<a href="https://colab.research.google.com/github/JamesKha/MovieRecommendation/blob/main/GeneralRecommendationsOnTags.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
tagsDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/tags.csv')
moviesDF = pd.read_csv('/content/drive/MyDrive/Miniproject Files/movies.csv')

#Recommending with tags

In [3]:
tagsAndGenresDataFrame = pd.merge(tagsDF, moviesDF, on='movieId')[['movieId','title', 'tag','genres']]
titlesAndTags = pd.DataFrame(tagsAndGenresDataFrame.groupby(['title']).tag.unique())
titlesAndTags = titlesAndTags.reset_index()
titlesAndTags['tag'] = titlesAndTags['tag'].apply(lambda x: ' '.join([str(i) for i in x]))
titlesAndTags['tag'] = titlesAndTags['tag'].str.lower()

In [4]:
titlesAndTags.head()

,title,tag
0,(500) Days of Summer (2009),artistic funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense
3,10 Things I Hate About You (1999),shakespeare sort of
4,101 Dalmatians (1996),dogs remake


In [5]:
tfidf = TfidfVectorizer(stop_words='english')
titlesAndTags['tag'] = titlesAndTags['tag'].fillna('')
tfidf_matrix = tfidf.fit_transform(titlesAndTags['tag'])

In [6]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [7]:
indices = pd.Series(titlesAndTags.index, index=titlesAndTags['title']).drop_duplicates()

In [8]:
indices

title
(500) Days of Summer (2009)             0
...And Justice for All (1979)           1
10 Cloverfield Lane (2016)              2
10 Things I Hate About You (1999)       3
101 Dalmatians (1996)                   4
                                     ... 
Zero Dark Thirty (2012)              1567
Zombieland (2009)                    1568
Zoolander (2001)                     1569
Zulu (1964)                          1570
eXistenZ (1999)                      1571
Length: 1572, dtype: int64

In [9]:
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))


    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


    sim_scores = sim_scores[1:11]


    movie_indices = [i[0] for i in sim_scores]


    return titlesAndTags[['title','tag']].iloc[movie_indices]

In [11]:
titlesAndTags

,title,tag
0,(500) Days of Summer (2009),artistic funny humorous inspiring intelligent ...
1,...And Justice for All (1979),lawyers
2,10 Cloverfield Lane (2016),creepy suspense
3,10 Things I Hate About You (1999),shakespeare sort of
4,101 Dalmatians (1996),dogs remake
...,...,...
1567,Zero Dark Thirty (2012),afghanistan american propaganda assassination ...
1568,Zombieland (2009),bill murray dark comedy emma stone funny jesse...
1569,Zoolander (2001),ben stiller comedy david bowie goofy mindless ...
1570,Zulu (1964),africa


In [12]:
get_recommendations('Step Brothers (2008)')

,title,tag
986,Old School (2003),comedy will ferrell
66,Anchorman 2: The Legend Continues (2013),comedy steve carell stupid but funny will ferrell
36,Airheads (1994),comedy
242,Chalet Girl (2011),comedy
146,"Big Lebowski, The (1998)",coen brothers black comedy bowling classic coe...
491,Game Night (2018),comedy funny rachel mcadams
67,Anchorman: The Legend of Ron Burgundy (2004),hilarious steve carell will ferrell stupid awe...
274,Clueless (1995),chick flick funny paul rudd quotable seen more...
713,Jumanji: Welcome to the Jungle (2017),action comedy dwayne johnson funny
1385,The Lego Batman Movie (2017),funny heartwarming


#Implementation with genres

In [13]:
movieAndGenres = tagsAndGenresDataFrame

In [14]:
movieAndGenres['tag'] =  movieAndGenres.groupby(['movieId','title'])['tag'].transform(lambda x: ' '.join(x))

In [15]:
movieAndGenres = movieAndGenres.drop_duplicates()

In [16]:
movieAndGenres

,movieId,title,tag,genres
0,60756,Step Brothers (2008),funny Highly quotable will ferrell comedy funn...,Comedy
8,89774,Warrior (2011),Boxing story MMA Tom Hardy,Drama
11,106782,"Wolf of Wall Street, The (2013)",drugs Leonardo DiCaprio Martin Scorsese Stock ...,Comedy|Crime|Drama
16,48516,"Departed, The (2006)",way too long Leonardo DiCaprio suspense twist ...,Crime|Drama|Thriller
26,431,Carlito's Way (1993),Al Pacino gangster mafia,Crime|Drama
...,...,...,...,...
3677,1948,Tom Jones (1963),British,Adventure|Comedy|Romance
3678,5694,Staying Alive (1983),70mm,Comedy|Drama|Musical
3679,6107,Night of the Shooting Stars (Notte di San Lore...,World War II,Drama|War
3680,7936,Shame (Skammen) (1968),austere,Drama|War


In [17]:
movieAndGenres['genresList'] = movieAndGenres['genres'].str.split("|")
movieAndGenres.drop('genres',axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
movieAndGenres['genresList'] = movieAndGenres['genresList'].apply(lambda x: ' '.join([str(i) for i in x]))
movieAndGenres['genresList'] = movieAndGenres['genresList'].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
movieAndGenres['year'] = movieAndGenres['title'].str.extract('.*\((.*)\).*')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
def create_soup(x):
    return ''.join(x['tag'].lower()) + ' ' + ''.join(x['genresList'])

In [21]:
movieAndGenres['soup'] = movieAndGenres.apply(create_soup, axis=1)
movieAndGenres.reset_index(drop=True,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#Word Soup


In [22]:
tfidf_matrix = tfidf.fit_transform(movieAndGenres['soup'])

In [23]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [24]:
movieAndGenresindices = pd.Series(movieAndGenres.index, index=movieAndGenres['title']).drop_duplicates()

In [25]:
movieAndGenresindices

title
Step Brothers (2008)                                                0
Warrior (2011)                                                      1
Wolf of Wall Street, The (2013)                                     2
Departed, The (2006)                                                3
Carlito's Way (1993)                                                4
                                                                 ... 
Tom Jones (1963)                                                 1567
Staying Alive (1983)                                             1568
Night of the Shooting Stars (Notte di San Lorenzo, La) (1982)    1569
Shame (Skammen) (1968)                                           1570
Hard-Boiled (Lat sau san taam) (1992)                            1571
Length: 1572, dtype: int64

In [26]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = movieAndGenresindices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))


    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


    sim_scores = sim_scores[1:11]


    movie_indices = [i[0] for i in sim_scores]


    return movieAndGenres[['title','soup']].iloc[movie_indices]

In [30]:
get_recommendations('Step Brothers (2008)')

,title,soup
234,Old School (2003),comedy will ferrell comedy
61,Anchorman 2: The Legend Continues (2013),comedy steve carell stupid but funny will ferr...
30,Anchorman: The Legend of Ron Burgundy (2004),hilarious steve carell will ferrell stupid awe...
1541,The Lego Batman Movie (2017),funny heartwarming action animation comedy
125,The DUFF (2015),funny high school comedy
1532,"Big Short, The (2015)",funny interesting witty drama
78,Game Night (2018),comedy funny rachel mcadams action comedy crim...
1509,Inside Llewyn Davis (2013),atmospheric cinematography depressing funny drama
230,Zoolander (2001),ben stiller comedy david bowie goofy mindless ...
1526,Kung Fury (2015),funny ridiculous silly action comedy fantasy s...


In [28]:
movieAndGenresindices

title
Step Brothers (2008)                                                0
Warrior (2011)                                                      1
Wolf of Wall Street, The (2013)                                     2
Departed, The (2006)                                                3
Carlito's Way (1993)                                                4
                                                                 ... 
Tom Jones (1963)                                                 1567
Staying Alive (1983)                                             1568
Night of the Shooting Stars (Notte di San Lorenzo, La) (1982)    1569
Shame (Skammen) (1968)                                           1570
Hard-Boiled (Lat sau san taam) (1992)                            1571
Length: 1572, dtype: int64